<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings
from bs4 import BeautifulSoup
import requests
import urllib.parse as req

In [2]:
def pd_read_sql_query(sql, conn):
    """ Функция преобразующая метода pd.read_sql_query(), поскольку этот метод
    сопровождался выводом предупреждения о поддержке объектов DBAPI2.
    
    Args: 
    sql - SQL-запрос, который необходимо выполнить 
    сonn - SQLAlchemy connectable, str или sqlite3 соединение
    """
    with warnings.catch_warnings():  # Отключаем warning
        warnings.simplefilter('ignore')
        return pd.read_sql_query(sql, conn)

In [3]:
# параметры подключения из юнита 1. Работа с базой данных из Python

In [4]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''SELECT 
                    COUNT(v.id) AS count
                FROM vacancies v
            '''

In [6]:
# результат запроса
df_3_1 = pd_read_sql_query(query_3_1, connection)
print('Количество вакансий в базе:', df_3_1.iloc[0, 0])

Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
# текст запроса
query_3_2 = f'''SELECT 
                    COUNT(e.id) AS count
                FROM employers e
            '''

In [8]:
# результат запроса
df_3_2 = pd_read_sql_query(query_3_2, connection)
print('Количество работодателей:', df_3_2.iloc[0, 0])

Количество работодателей: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
# текст запроса
query_3_3 = f'''SELECT 
                    COUNT(a.id) AS count
                FROM areas a
            '''

In [10]:
# результат запроса
df_3_3 = pd_read_sql_query(query_3_3, connection)
print('Количество регионов:', df_3_3.iloc[0, 0])

Количество регионов: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
# текст запроса
query_3_4 = f'''SELECT 
                    COUNT(i.id) AS count
                FROM industries i
            '''

In [12]:
# результат запроса
df_3_4 = pd_read_sql_query(query_3_4, connection)
print('Количество сфер деятельности:', df_3_4.iloc[0, 0])

Количество сфер деятельности: 294


***

#### Выводы по предварительному анализу данных

1. Количество вакансий (49197) превышает количество работодателей (23501). Это может указывать на то, что некоторые работодатели публикуют несколько вакансий.

2. Количество регионов: 1362. Это означает,  что «регионом» в базе называются не только регионы РФ, но и города, страны. 

3. Количество сфер деятельности (294) также сравнительно невелико по сравнению с числом вакансий и работодателей. Это может указывать на то, что в базе данных представлены в основном популярные и широко распространенные сферы деятельности, а множество мелких и узкоспециализированных сфер не представлены.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query_4_1 = f'''SELECT 
                    a.name area,
                    COUNT(v.id) cnt
                FROM areas a 
                    LEFT JOIN vacancies v ON a.id = v.area_id
                GROUP BY area
                ORDER BY cnt DESC 
                LIMIT 5
            '''

In [14]:
# результат запроса
df_4_1 = pd_read_sql_query(query_4_1, connection)
display(df_4_1)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query_4_2 = f'''SELECT 
                    COUNT(v.id) cnt
                FROM vacancies v 
                WHERE salary_from IS NOT NULL  OR salary_to IS NOT NULL
            '''

In [16]:
# результат запроса
df_4_2 = pd_read_sql_query(query_4_2, connection)
print('Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой:', df_4_2.iloc[0, 0])

Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query_4_3 = f'''SELECT 
                    ROUND(AVG(v.salary_from)) lower_salary, -- средние значения для нижней
                    ROUND(AVG(v.salary_to)) upper_salary    -- и верхней границы зарплатной вилки
                FROM vacancies v 
            '''

In [18]:
# результат запроса
df_4_3 = pd_read_sql_query(query_4_3, connection)
display(df_4_3)

,lower_salary,upper_salary
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query_4_4 = f'''SELECT 
                    v.schedule schedule,
                    v.employment employment,
                    COUNT(v.id)  cnt   -- количество вакансий
                FROM vacancies v 
                GROUP BY schedule, employment
                ORDER BY 3 DESC 
                OFFSET 1       --Какая пара находится на втором месте по популярности?
                LIMIT 1 
            '''

In [20]:
# результат запроса
df_4_4 = pd_read_sql_query(query_4_4, connection)

# Какая пара находится на втором месте по популярности?
display(df_4_4)

,schedule,employment,cnt
0,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query_4_5 = f'''
                SELECT 
                    v.experience experience
                FROM vacancies v
                GROUP BY experience
                ORDER BY COUNT(v.id) --в порядке возрастания количества вакансий
            '''

In [22]:
# результат запроса
df_4_5 = pd_read_sql_query(query_4_5, connection)
display(df_4_5)

,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


#### Дополнительное исследование

6. Посчитайте среднюю зарплату для всех вакансий в базе данных (таблица vacancies). В зарплатной вилке указана нижняя и верхняя границы оплаты труда. Найдите среднее значение этой вилки для всех вакансий.


In [23]:
# текст запроса
query_4_6 = f'''
                SELECT
                    ROUND(AVG((salary_from + salary_to) / 2)) AS average_salary
                FROM vacancies
            '''

In [24]:
# результат запроса
df_4_6 = pd_read_sql_query(query_4_6, connection)
print('Средняя зарплата для всех вакансий в базе данных:', df_4_6.iloc [0, 0])

Средняя зарплата для всех вакансий в базе данных: 89074.0


***

####  Выводы по детальному анализу вакансий

1. Самые популярные города по количеству вакансий - Москва, Санкт-Петербург, Минск, Новосибирск и Алматы, т.е. города из топ 3 по населению в РФ и столицы Беларуси и Казахстана. Это свидетельствует о том, что эти города являются крупным рынком труда с высоким спросом на различных специалистов.

2. Примерно у половины вакансий (24073) заполнено хотя бы одно из полей с зарплатой.

3. Средние значения для нижней и верхней границы зарплатной вилки составляют 71065 и 110537 рублей соответственно. Это позволяет оценить средний уровень зарплат на рынке труда в представленных вакансиях.

4. Самый распространенный тип занятости - "Полная занятость", и среди них особенно популярны форматы "Полный день" и "Удаленная работа".

5. Вакансий для специалистов с опытом от 3 до 6 лет больше всего, а вакансий для специалистов с опытом от 3 до 6 лет в два раза больше, чем вакансий совсем без опыта
 

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [25]:
# текст запроса
query_5_1 = f'''
               SELECT 
                    e.name employer,   -- работодатели
                    COUNT(v.id) cnt    -- количество вакансий
                FROM employers e 
                    LEFT JOIN vacancies v ON e.id = v.employer_id
                GROUP BY employer
                ORDER BY COUNT(v.id) DESC 
                LIMIT 5                -- топ 5

            '''

In [26]:
# результат запроса
df_5_1 = pd_read_sql_query(query_5_1, connection)
display(df_5_1)

,employer,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [27]:
# текст запроса
query_5_2 = f'''
               SELECT 
                    a.name area_name,  -- регион
                    COUNT(e.id) employers,   -- количество работодателнй
                    COUNT(v.id) vacancies    -- количество вакансий
               FROM areas a 
                   LEFT JOIN employers e ON a.id = e.area
                   LEFT JOIN vacancies v ON a.id = v.area_id
               GROUP BY area_name
               HAVING COUNT(v.id) = 0
               ORDER BY COUNT(e.id) DESC
               LIMIT 5
            '''

In [28]:
# результат запроса
df_5_2 = pd_read_sql_query(query_5_2, connection)
display(df_5_2)
print('Регион в котором больше всего работодателей:', df_5_2.iloc [0, 0])

,area_name,employers,vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


Регион в котором больше всего работодателей: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [29]:
# текст запроса
query_5_3 = f'''
               SELECT 
                   e.name employer,                   -- работодатель
                   COUNT(DISTINCT v.area_id) areas    -- количество регионов
                FROM  employers e
                    LEFT JOIN vacancies v ON e.id = v.employer_id
                GROUP BY employer
                ORDER BY areas DESC
                LIMIT 1
            '''

In [30]:
# результат запроса
df_5_3 = pd_read_sql_query(query_5_3, connection)
display(df_5_3)

,employer,areas
0,Яндекс,181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [31]:
# текст запроса
query_5_4 = f'''
               SELECT 
                   COUNT(e.id) employers
                FROM  employers e
                    LEFT JOIN employers_industries ei ON e.id = ei.employer_id
                WHERE ei.industry_id IS NULL
            '''

In [32]:
# результат запроса
df_5_4 = pd_read_sql_query(query_5_4, connection)
print('Количество работодателей, у которых не указана сфера деятельности:', df_5_4.iloc [0, 0])

Количество работодателей, у которых не указана сфера деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [33]:
# текст запроса
query_5_5 = f'''
                SELECT 
                    e.name employer
                   -- COUNT(ei.industry_id) num_industries
                FROM employers e 
                    LEFT JOIN employers_industries ei ON e.id = ei.employer_id
                GROUP BY e.name
                HAVING COUNT(ei.industry_id) = 4
                ORDER BY employer
                OFFSET 2 
                LIMIT 1
            '''

In [34]:
# результат запроса
df_5_5 = pd_read_sql_query(query_5_5, connection)
print('Компания на третьем месте в списке, у которых указано четыре сферы деятельности:', df_5_5.iloc[0, 0])

Компания на третьем месте в списке, у которых указано четыре сферы деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана "Разработка программного обеспечения".


In [35]:
# текст запроса
query_5_6 = f'''
                SELECT 
                    COUNT(ei.employer_id) num_employers
                FROM employers_industries ei 
                   LEFT JOIN industries i ON ei.industry_id = i.id 
                WHERE i.name = 'Разработка программного обеспечения'
            '''

In [36]:
# результат запроса
df_5_6 = pd_read_sql_query(query_5_6, connection)
print('Количество работодателей, где в качестве сферы деятельности указана \
"Разработка программного обеспечения":', df_5_6.iloc [0, 0])

Количество работодателей, где в качестве сферы деятельности указана "Разработка программного обеспечения": 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [37]:
# Код для получения списка городов-милионников по ссылке из Википедии
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%' +\
      'D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%' +\
      'D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

# Отправляем GET-запрос к указанному URL и получаем ответ (HTML-код страницы)
response = requests.get(url)

# Создаем объект BeautifulSoup 
# response.text содержит содержимое ответа (HTML-код)
soup = BeautifulSoup(response.text, 'html.parser')

# Используем селектор, чтобы найти все строки (теги <tr>) во второй таблице (<table>) на странице.
# С помощью генератора списка извлекаем названия городов-милионников из атрибута 
# 'title' элементов <a> в каждой строке.
# С помощью [1:] исключаем первую строку, которая обычно содержит заголовки столбцов таблицы.
# Затем преобразуем полученный список в кортеж (tuple).
cities = tuple([row.find('a')['title']
               for row in soup.select('table:nth-of-type(2) tr')[1:]])

print(cities)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [38]:
# текст запроса
query_5_7 = f'''
                WITH temp AS (SELECT 
                    a.name area_name,
                    COUNT(v.id) cnt
                FROM areas a 
                    LEFT JOIN vacancies v  ON a.id = v.area_id 
                    LEFT JOIN employers e  ON v.employer_id = e.id
                WHERE e.name = 'Яндекс'
                GROUP BY a.name
                HAVING a.name IN {cities}
                ORDER BY cnt DESC)
                SELECT * FROM temp
                UNION ALL 
                SELECT 'Total', SUM(cnt) FROM temp
             '''

In [39]:
# результат запроса
df_5_7 = pd_read_sql_query(query_5_7, connection)
display(df_5_7)

,area_name,cnt
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


#### Дополнительное исследование

8. Найдите топ-5 работодателей, предлагающих самые высокие зарплаты вакансий с указанием зарплатной вилки (salary_from и salary_to). Выведите название работодателя и среднее значение верхней границы зарплатной вилки.

In [40]:
# текст запроса
query_5_8 = f'''
                SELECT 
                    e.name AS employer,
                    ROUND(AVG(v.salary_to)) AS average_upper_salary
                FROM employers e
                    LEFT JOIN vacancies v ON e.id = v.employer_id
                WHERE v.salary_to IS NOT NULL
                GROUP BY e.name
                ORDER BY AVG(v.salary_to) DESC
                LIMIT 10
             '''

In [41]:
# результат запроса
df_5_8 = pd_read_sql_query(query_5_8, connection)
display(df_5_8)

,employer,average_upper_salary
0,ЦТП,950000.0
1,RuWork,935176.0
2,Агентство недвижимости ЛЕТО,750000.0
3,Петухова Алия,712200.0
4,Итон,700000.0
5,BeRecruited,600000.0
6,Шеф Маркет,600000.0
7,Пикабу,600000.0
8,ШтафунСофт,600000.0
9,BrainShells,584485.0


***

#### Выводы по анализу работодателей

1. Топ-5 работодателей по количеству вакансий: "Яндекс", "Ростелеком", "Тинькофф", "СБЕР", "Газпром нефть". Эти компании предоставляют большое количество вакансий и, вероятно, являются крупными игроками на рынке труда.

2. Низкая активность в некоторых регионах: топ-5 регионов с наибольшим количеством работодателей, но без вакансий (Россия, Казахстан, Московская область, Краснодарский край, Беларусь), может свидетельствовать о низкой активности или конкретных условиях рынка труда в этих регионах. Возможно, в них есть определенные структурные особенности, которые препятствуют размещению вакансий.

3. Лидеры по числу регионов, в которых публикуются вакансии: "Яндекс", "Ростелеком", "Спецремонт", "Поляков Денис Иванович", "ООО ЕФИН". Эти компании имеют наибольшее присутствие в различных регионах и, возможно, имеют широкий охват деятельности.

4. У 8419 (36%) работодателей не указана сфера деятельности. Это может затруднить соискателям поиск вакансий, а также затруднить анализ рынка труда по определенным сферам деятельности.

5. У большого количества работодателей указано от 1 до 5 сфер деятельности. Это может свидетельствовать о разнообразии вакансий и разнообразии требуемых навыков на рынке труда.

6. 3553 (15%) работодателя указали "Разработку программного обеспечения" как сферу деятельности. Это может указывать на высокий спрос на IT-специалистов и связанных с ними вакансий.

7. Компания "Яндекс" представлена во многих регионах-миллионниках, с общим количеством вакансий в этих регионах равным 485. Это может свидетельствовать о широкой географии деятельности этой компании.

8. Процент вакансий, предлагаемых топ-5 работодателями по количеству вакансий, составляет примерно 11.3% от общего числа вакансий в базе данных. Это может указывать на значительное влияние этих компаний на рынок труда и предоставляемые ими возможности для соискателей.

9. Топ-10 работодателей с самыми высокими предлагаемыми зарплатами не пресекается с топ-10 работодателей по количеству вакансий. Можно сделать вывод, что в более известные компании соискатели согласны приходить на меньшую зарплату. 


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [42]:
# текст запроса
query_6_1 = f'''
                SELECT 
                    COUNT(v.id) cnt
                FROM vacancies v
                WHERE lower(v.name) LIKE '%data%' OR lower(v.name) LIKE '%данн%'
             '''

In [43]:
# результат запроса
df_6_1 = pd_read_sql_query(query_6_1, connection)
print('Вакансий, имеющих отношение к данным:', df_6_1.iloc[0, 0])


Вакансий, имеющих отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [44]:
# текст запроса
query_6_2_1 = f'''
                SELECT 
                    COUNT(v.id) cnt
                FROM vacancies v
                WHERE (lower(v.name) LIKE '%data scientist%' 
                    OR lower(v.name) LIKE '%data science%'
                    OR lower(v.name) LIKE '%исследователь данных%'
                    OR lower(v.name) LIKE '%machine learning%'
                    OR lower(v.name) LIKE '%машинн%обучен%'
                    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%'))
                    AND (lower(v.name) LIKE '%junior%' 
                       OR v.experience LIKE '%Нет опыта%'
                       OR v.employment LIKE '%Стажировка%')
            '''

In [45]:
# результат запроса
df_6_2_1 = pd_read_sql_query(query_6_2_1, connection)
print('Количество вакансий для начинающего дата-сайентиста:', df_6_2_1.iloc[0, 0])

Количество вакансий для начинающего дата-сайентиста: 51


In [46]:
# Более лаконичный варинат текста запроса
query_6_2_2 = f'''
                  SELECT
                      COUNT(v.id) cnt
                  FROM vacancies v
                  WHERE
                  (name ILIKE ANY (ARRAY['%data scientist%',                       
                                         '%data science%', 
                                         '%исследователь данных%', 
                                         '%machine learning%', 
                                         '%машинн%обучен%']) 
                   OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%'))
                   AND (name ILIKE '%junior%' 
                    OR experience ILIKE '%Нет опыта%'
                    OR employment ILIKE '%Стажировка%') 
               '''

In [47]:
# результат запроса
df_6_2_2 = pd_read_sql_query(query_6_2_2, connection)
print('Количество вакансий для начинающего дата-сайентиста:', df_6_2_2.iloc[0, 0])

Количество вакансий для начинающего дата-сайентиста: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [48]:
# текст запроса
query_6_3 = f'''
                SELECT
                    COUNT(v.id) cnt
                FROM vacancies v
                WHERE
                (name ILIKE ANY (ARRAY['%data scientist%',                       
                                       '%data science%', 
                                       '%исследователь данных%', 
                                       '%machine learning%', 
                                       '%машинн%обучен%']) 
                 OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%'))
                 AND (v.key_skills ILIKE ANY (ARRAY['%SQL%', '%postgres%']))
            '''

In [49]:
# результат запроса
df_6_3 = pd_read_sql_query(query_6_3, connection)
print('Количество вакансий для DS, в которых в качестве \
ключевого навыка указан SQL или postgres:', df_6_3.iloc[0, 0])

Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [50]:
# текст запроса
query_6_4 = f'''
                SELECT
                    COUNT(v.id) cnt
                FROM vacancies v
                WHERE
                (name ILIKE ANY (ARRAY['%data scientist%',                       
                                       '%data science%', 
                                       '%исследователь данных%', 
                                       '%machine learning%', 
                                       '%машинн%обучен%']) 
                 OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%'))
                 AND v.key_skills ILIKE '%Python%'
            '''

In [51]:
# результат запроса
df_6_4 = pd_read_sql_query(query_6_4, connection)
print('Количество вакансий для DS, в которых в качестве \
ключевого навыка указан Python:', df_6_4.iloc[0, 0])

Количество вакансий для DS, в которых в качестве ключевого навыка указан Python: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [52]:
# текст запроса
query_6_5 = f'''
                SELECT
                    /*
                    С помощью функции regexp_split_to_array преобразуем строку с навыками в массив строк, 
                    содержащий отдельные ключевые навыки.
                    Затем, с помощью функции ARRAY_LENGTH(array, 1) получаем  длину массива и вычисляем 
                    среднее количество навыков.
                    */
                     ROUND(AVG(ARRAY_LENGTH(regexp_split_to_array(v.key_skills, '\t'), 1)), 2) AS avg_skills
                FROM vacancies v
                WHERE (name ILIKE ANY (ARRAY['%data scientist%',                       
                                             '%data science%', 
                                             '%исследователь данных%', 
                                             '%machine learning%', 
                                             '%машинн%обучен%']) 
                                   OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%'))
            '''

In [53]:
# результат запроса
df_6_5 = pd_read_sql_query(query_6_5, connection)
print('Количество ключевых навыков в среднем в вакансиях для DS:', df_6_5.iloc[0, 0])

Количество ключевых навыков в среднем в вакансиях для DS: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [54]:
# текст запроса
query_6_6 = f'''
                SELECT
                    DISTINCT v.experience experience,          
                    ROUND(AVG(coalesce((v.salary_from + v.salary_to)/2, salary_from, salary_to))) avg_salary
                FROM vacancies v
                WHERE (name ILIKE ANY (ARRAY['%data scientist%',                       
                                             '%data science%', 
                                             '%исследователь данных%', 
                                             '%machine learning%', 
                                             '%машинн%обучен%']) 
                    OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%'))
                    AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
                GROUP BY experience
            '''

In [55]:
# результат запроса
df_6_6 = pd_read_sql_query(query_6_6, connection)
display(df_6_6)
print('Дата-сайентист с опытом работы от 3 до 6 лет может \
рассчитывать в среднем на зарплату:', df_6_6.iloc[2, 1])

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


Дата-сайентист с опытом работы от 3 до 6 лет может рассчитывать в среднем на зарплату: 243115.0


#### Дополнительное исследование

7. Как распределены вакансии для дата-сайентистов по различным регионам? Какой регион предлагает наибольшее количество вакансий? Напишите SQL-запрос, который вернет количество вакансий для каждого региона.

In [56]:
# текст запроса
query_6_7 = f'''
                SELECT
                    a.name AS region,
                    COUNT(v.id) AS cnt
                FROM vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id
                WHERE (v.name ILIKE ANY (ARRAY['%data scientist%',                       
                                               '%data science%', 
                                               '%исследователь данных%', 
                                               '%machine learning%', 
                                               '%машинн%обучен%']) 
                     OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%'))
                GROUP BY a.name
                ORDER BY cnt DESC
                LIMIT 10
            '''

In [57]:
# результат запроса 
df_6_7 = pd_read_sql_query(query_6_7, connection)

#количество вакансий для дата-сайентистов в различных регионах
display(df_6_7)

,region,cnt
0,Москва,217
1,Санкт-Петербург,64
2,Новосибирск,23
3,Нижний Новгород,20
4,Казань,16
5,Алматы,16
6,Минск,9
7,Томск,8
8,Екатеринбург,7
9,Армения,6


8. Какие сферы деятельности (industries) наиболее популярны среди работодателей, ищущих дата-сайентистов? Какое количество вакансий предлагается для каждой сферы деятельности? Напишите SQL-запрос, который вернет список сфер деятельности и количество вакансий для каждой из них, отсортированных по убыванию количества вакансий.


In [58]:
# текст запроса
query_6_8 = f'''
                SELECT
                    i.name AS industry,
                    COUNT(v.id) AS cnt
                FROM vacancies v
                    LEFT JOIN employers_industries ei ON v.employer_id = ei.employer_id
                    LEFT JOIN industries i ON ei.industry_id = i.id
                WHERE (v.name ILIKE ANY (ARRAY['%data scientist%',                       
                                               '%data science%', 
                                               '%исследователь данных%', 
                                               '%machine learning%', 
                                               '%машинн%обучен%']) 
                     OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%'))
                GROUP BY i.name
                ORDER BY cnt DESC
                LIMIT 10
            '''

In [59]:
# результат запроса 
df_6_8 = pd_read_sql_query(query_6_8, connection)

# список сфер деятельности и количество вакансий для каждой из них
display(df_6_8)

,industry,cnt
0,Разработка программного обеспечения,148
1,"Системная интеграция, автоматизации технологи...",139
2,Банк,92
3,"Интернет-компания (поисковики, платежные систе...",79
4,None,62
5,Мобильная связь,23
6,"Маркетинговые, рекламные, BTL, дизайнерские, E...",22
7,"Управляющая, инвестиционная компания (управлен...",21
8,Консалтинговые услуги,16
9,Фиксированная связь,13


9. Сколько вакансий для дата-сайентистов требуют знание английского языка (ключевой навык или требование в описании)?


In [60]:
# текст запроса
query_6_9 = f'''
                SELECT 
                    COUNT(DISTINCT v.id) AS num_vacancies
                FROM vacancies v
                WHERE (v.name ILIKE ANY (ARRAY['%data scientist%',                       
                                               '%data science%', 
                                               '%исследователь данных%', 
                                               '%machine learning%', 
                                               '%машинн%обучен%']) 
                     OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%'))
                     AND (v.key_skills ILIKE '%английск%' OR v.name ILIKE '%английск%')
            '''

In [61]:
# результат запроса
df_6_9 = pd_read_sql_query(query_6_9, connection)
print('Количество вакансий для дата-сайентистов, в которых \
требуется знание английского языка:', df_6_9.iloc[0, 0])

Количество вакансий для дата-сайентистов, в которых требуется знание английского языка: 48


***

#### Выводы по предметному анализу

1. Востребованность. Вакансий, связанных с данными, в целом достаточно много 1771 вакансия (3.6% от всего количества вакансий). Это свидетельствует о высокой востребованности специалистов в данной области.

2. Уровень входа. Вакансий для начинающих дата-сайентистов относительно немного (51 вакансия), что может означать, что рынок предпочитает специалистов с опытом работы и новичкам найти работу будет нелегко из-за высокой конкуренции. 

3. Требования к навыкам. В ключевых навыках наиболее популярны Python (351 вакансия) и SQL (201 вакансия) . Это подтверждает важность данных языков программирования для работы с данными и анализа.

4. Уровень навыков: В среднем в вакансиях указывают около 6.41 ключевого навыка, что говорит о высоких требованиях работодателей к компетенциям дата-сайентистов.

5. Зарплата: Средняя зарплата для дата-сайентистов с опытом работы от 3 до 6 лет составляет около 243115 рублей. Это значительно выше средней зарплата для всех вакансий в базе данных - 89074 рублей. В вакансиях с опытом более 6 лет не указывают уровень зарплат. 

6. Большинство вакансий сосредоточено в городах-миллиониках (лидеры Москва, Санкт-Петербург)

7. Сферы деятельности: Наиболее популярными сферами деятельности, ищущими дата-сайентистов, являются Разработка ПО, банковская сфера, системная интеграция и автоматизация, связь, интернет-компании, инвестиции, консалтинг.  У некторых копаний сфера деятельности не указана - 5 место. 

8. Количество вакансий, где требуется знание английского около 3%. 


In [62]:
# Закрываем соединение
connection.close()

# Общий вывод по проекту

Рынок труда в области дата-сайенса является активным, существует значительное количество вакансий для специалистов этой области, что свидетельствует о их высокой востребованности на рынке.

Наличие знания английского языка востребовано, но требования к нему не являются обязательными для большинства вакансий, что может быть важной информацией для соискателей.

Требования к навыкам дата-сайентистов высоки, в среднем в вакансиях указывается около 6 ключевых навыков.

Уровень опыта играет важную роль в требованиях к соискателям. Вакансии для начинающих дата-сайентистов составляют небольшую часть, а средние уровни опыта (от 3 до 6 лет) наиболее популярны.

Средняя зарплата для дата-сайентистов с опытом работы от 3 до 6 лет является привлекательной и значительно выше средней зарплаты по всем вакансиям.

Некоторые известные компании активно представлены на рынке труда с большим количеством вакансий, их присутствие может оказывать сильное влияние на рынок.

Города-миллионники являются основными регионами с высокой активностью вакансий.

Некоторые сферы деятельности показывают более высокую активность и востребованность специалистов дата-сайенса.

В целом, рынок труда для дата-сайентистов представляет собой перспективную область с высоким уровнем востребованности и привлекательными зарплатами, особенно для специалистов с опытом работы от 3 до 6 лет. Знание английского языка может быть дополнительным преимуществом для соискателей. Также рекомендуется активно изучать и развивать ключевые навыки, востребованные на рынке труда, чтобы быть более конкурентоспособными на этом растущем рынке.